# Mutual funds vs. Index drawdowns

Do active managers reduce drawdowns during downturns?

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

indices <- Indices()
mfi <- MutualFundsIndia()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.1
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘PerformanceAnalytics’

The following object is masked from ‘package:graphics’:

    legend

Registering fonts with R

Attaching package: ‘gridExtra’

The following object is masked from ‘package:dplyr’:

    combine


Attaching package

2018 was a bad year. So lets look at the subsequent return profile of the top 5 midcap funds entring 2018 through June-2019.

In [2]:
lastAumDate <- (mfi$AumSchemewise() %>%
    filter(PERIOD <= as.Date('2018-01-01')) %>%
    summarize(MAX_TS = max(PERIOD)) %>%
    collect())$MAX_TS[1]

lastMetaDate <- (mfi$Meta() %>%
    filter(AS_OF >= lastAumDate) %>%
    summarize(MIN_TS = min(AS_OF)) %>%
    collect())$MIN_TS[1]

print(lastAumDate)
print(lastMetaDate)

top5Aum <- mfi$Meta() %>% 
    inner_join(mfi$AumSchemewise(), by="SCHEME_CODE") %>%
    select(SCHEME_CODE, SCHEME_NAME, AVG_AUM_WO_FOFD) %>%
    filter(PERIOD == lastSwDate & AS_OF == lastMetaDate & CATEGORY == 'Mid-Cap') %>%
    top_n(n = 5, AVG_AUM_WO_FOFD)

print(top5Aum)

Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

[1] "2017-12-01"
[1] "2018-03-24"


ERROR: Error: <SQL> 'SELECT TOP(11) "SCHEME_CODE", "SCHEME_NAME", "AVG_AUM_WO_FOFD"
FROM (SELECT TOP 100 PERCENT "SCHEME_CODE", "SCHEME_NAME", "AVG_AUM_WO_FOFD", RANK() OVER (ORDER BY "AVG_AUM_WO_FOFD" DESC) AS "zzz5"
FROM (SELECT TOP 100 PERCENT *
FROM (SELECT TOP 100 PERCENT "SCHEME_CODE", "SCHEME_NAME", "AVG_AUM_WO_FOFD"
FROM (SELECT "LHS"."SCHEME_CODE" AS "SCHEME_CODE", "LHS"."AS_OF" AS "AS_OF", "LHS"."BENCH_ORIG" AS "BENCH_ORIG", "LHS"."BENCH_NOW" AS "BENCH_NOW", "LHS"."CATEGORY" AS "CATEGORY", "LHS"."EXPENSE" AS "EXPENSE", "LHS"."EXPENSE_RATIO" AS "EXPENSE_RATIO", "LHS"."TURNOVER_RATIO" AS "TURNOVER_RATIO", "RHS"."PERIOD" AS "PERIOD", "RHS"."SCHEME_NAME" AS "SCHEME_NAME", "RHS"."AVG_AUM_WO_FOFD" AS "AVG_AUM_WO_FOFD", "RHS"."AVG_AUM_FOFD" AS "AVG_AUM_FOFD"
FROM (SELECT TOP 100 PERCENT "SCHEME_CODE", "AS_OF", "BENCH_ORIG", "BENCH_NOW", "CATEGORY", "EXPENSE", "EXPENSE_RATIO", "TURNOVER_RATIO"
FROM "MF_META2") "LHS"
INNER JOIN (SELECT TOP 100 PERCENT "PERIOD", "AMFI_CODE" AS "SCHEME_

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)